In [1]:
!pip install pandas
!pip install openpyxl
!pip install matplotlib
!pip install seaborn
!pip install scipy
!pip install statsmodels
!pip install xlsxwriter



In [4]:
#delete rows ending with vHPC in 'Image' column
import pandas as pd

# 读取原始Excel文件
df = pd.read_excel('/Users/sc17237/Desktop/TRAP2 Ket data analysis/ket trap cell count.xlsx')

# 使用正则表达式筛选出 'Image' 列中以 'vHPC' 结尾的行
df = df[~df['Image'].str.contains('vHPC$', regex=True)]

# 创建新的Excel文件并将数据写入
df.to_excel('/Users/sc17237/Desktop/TRAP2 Ket data analysis/ket trap cell count 去掉vHPC.xlsx', index=False)

print('操作完成，已经生成新的Excel文件。')



操作完成，已经生成新的Excel文件。


In [7]:
#结合ACA，PL，ILA统一为PFC
import pandas as pd

# 1. 读取Excel文件
file_path = '/Users/sc17237/Desktop/TRAP2 Ket data analysis/ket trap and dapi cell count 去掉vHPC.xlsx'
df = pd.read_excel(file_path)

# 2. 替换Area Name中的'ACA'，'ILA'，'PL'为'PFC'
df['Area Name'] = df['Area Name'].replace(['ACA', 'ILA', 'PL'], 'PFC')

# 3. 对拥有相同Image的Area Name为'PFC'的数据进行累加
agg_cols = ['Area', 'TRAP Cell Count', 'DAPI Cell Count']
grouped = df[df['Area Name'] == 'PFC'].groupby(['Image']).agg({col: 'sum' for col in agg_cols}).reset_index()
grouped['Area Name'] = 'PFC'

# 4. 保留其他的Area Name数据不变，并与上一步的结果进行拼接
result = pd.concat([df[df['Area Name'] != 'PFC'], grouped], ignore_index=True)

# 5. 保存到一个新的Excel文件
new_file_path = '/Users/sc17237/Desktop/TRAP2 Ket data analysis/processed_data.xlsx'
result.to_excel(new_file_path, index=False)

print(f"Processed data saved to {new_file_path}")



Processed data saved to /Users/sc17237/Desktop/TRAP2 Ket data analysis/processed_data.xlsx


In [9]:
#add treatment group to each animal

import pandas as pd
import re

# 1. 读取Excel文件
file_path = "/Users/sc17237/Desktop/TRAP2 Ket data analysis/ket trap and dapi cell count 去掉vHPC pfc合并.xlsx"
df = pd.read_excel(file_path)

# 2. 添加新的'Treatment Group'列
# 先创建一个函数来根据Image列确定Treatment Group
def assign_treatment_group(image):
    # 使用正则表达式提取数字部分
    match = re.search(r"MC2_(\d+)", image)
    if match:
        num = int(match.group(1))
        if num in [2, 5, 31, 32]:
            return 'Vehicle'
        elif num in [3, 21, 22, 30]:
            return '1 mg'
        elif num in [4, 19, 20, 29]:
            return '5 mg'
        else:
            return 'Unknown'  # 对于未指定的数字，返回'Unknown'
    return 'Unknown'

# 应用该函数到每一行
df['Treatment Group'] = df['Image'].apply(assign_treatment_group)

# 3. 保存为新的Excel文档
output_path = "/Users/sc17237/Desktop/TRAP2 Ket data analysis/processed_ket_dapi_cell_count.xlsx"
df.to_excel(output_path, index=False)



In [11]:
#sorting the file by brain area
with pd.ExcelWriter('/Users/sc17237/Desktop/TRAP2 Ket data analysis/ket trap and dapi cell count 去掉vHPC pfc合并 加TG.xlsx') as writer:
    for area_name, data in df.groupby('Area Name'):
        data.to_excel(writer, sheet_name=area_name, index=False)


In [14]:
#计算cell density per mm2

import pandas as pd

# 读取Excel文件
file_path = "/Users/sc17237/Desktop/TRAP2 Ket dapi only data analysis/sorted by area name.xlsx"
xls = pd.ExcelFile(file_path)

# 获取所有sheet的名字
sheet_names = xls.sheet_names

# 创建一个空的dictionary来保存处理后的数据
processed_data = {}

# 遍历每一个sheet进行处理
for sheet in sheet_names:
    df = pd.read_excel(file_path, sheet_name=sheet)
    
    # 将Area列的数据从μm2转换为mm2
    df['Area'] = df['Area'] / 1e6
    
    # 计算TRAP cell density并添加新的列
    df['TRAPed cell per mm2'] = df['TRAP Cell Count'] / df['Area']
    
    # 保存处理后的数据到dictionary中
    processed_data[sheet] = df

# 将处理后的数据保存到新的Excel文件中
output_path = "/Users/sc17237/Desktop/TRAP2 Ket dapi only data analysis/processed_ket_dapi_cell_count_density_mm2.xlsx"
with pd.ExcelWriter(output_path) as writer:
    for sheet, data in processed_data.items():
        data.to_excel(writer, sheet_name=sheet, index=False)


In [16]:
#计算cell percentage

import pandas as pd

# 读取Excel文件
file_path = "/Users/sc17237/Desktop/TRAP2 Ket dapi only data analysis/sorted by area name.xlsx"
xls = pd.ExcelFile(file_path)

# 获取所有sheet的名字
sheet_names = xls.sheet_names

# 创建一个空的dictionary来保存处理后的数据
processed_data = {}

# 遍历每一个sheet进行处理
for sheet in sheet_names:
    df = pd.read_excel(file_path, sheet_name=sheet)
    
    # 计算TRAP/DAPI percentage并添加新的列
    df['trap/dapi percentage'] = (df['TRAP Cell Count'] / df['DAPI Cell Count']) * 100  # 乘以100以表示百分比
    
    # 保存处理后的数据到dictionary中
    processed_data[sheet] = df

# 将处理后的数据保存到新的Excel文件中
output_path = "/Users/sc17237/Desktop/TRAP2 Ket dapi only data analysis/processed_trap_dapi_percentage.xlsx"
with pd.ExcelWriter(output_path) as writer:
    for sheet, data in processed_data.items():
        data.to_excel(writer, sheet_name=sheet, index=False)


In [20]:
#把结果转换成prism友好的格式

import pandas as pd

# 读取Excel文件
file_path = "/Users/sc17237/Desktop/TRAP2 Ket dapi only data analysis/cell percentage.xlsx"
xls = pd.ExcelFile(file_path)

# 获取所有sheet的名字
sheet_names = xls.sheet_names

# 创建一个空的dictionary来保存处理后的数据
processed_data = {}

# 遍历每一个sheet进行处理
for sheet in sheet_names:
    df = pd.read_excel(file_path, sheet_name=sheet)
    
    # 使用pivot来进行转置操作
    df_pivot = df.pivot(index='Image', columns='Treatment Group', values='trap/dapi percentage')
    
    # 确保列的顺序是Vehicle, 1 mg, 5 mg
    df_pivot = df_pivot[['Vehicle', '1 mg', '5 mg']]
    
    # 保存处理后的数据到dictionary中
    processed_data[sheet] = df_pivot

# 将处理后的数据保存到新的Excel文件中
output_path = "/Users/sc17237/Desktop/TRAP2 Ket dapi only data analysis/cell percentage prism.xlsx"
with pd.ExcelWriter(output_path) as writer:
    for sheet, data in processed_data.items():
        data.to_excel(writer, sheet_name=sheet, index=True)

